In [ ]:
import sys
import os
from pathlib import Path

# Añadir src al path
# El notebook está en: /portfolio/01_fraud_neuromorphic/notebooks/
# src está en: /portfolio/01_fraud_neuromorphic/src/
notebook_dir = Path.cwd()

# Comprobar si estamos en el directorio raíz del proyecto o en notebooks
if (notebook_dir / 'portfolio' / '01_fraud_neuromorphic' / 'src').exists():
    # Estamos en la raíz del proyecto (Project-Neuromorfico-X)
    src_path = notebook_dir / 'portfolio' / '01_fraud_neuromorphic' / 'src'
elif (notebook_dir.parent / 'src').exists():
    # Estamos en notebooks/
    src_path = notebook_dir.parent / 'src'
elif (notebook_dir / 'src').exists():
    # Estamos en 01_fraud_neuromorphic/
    src_path = notebook_dir / 'src'
else:
    src_path = None

if src_path and src_path.exists():
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
        print(f"✅ Directorio src añadido: {src_path}")
else:
    print(f"⚠️ ¡Directorio src no encontrado!")
    print(f"  Directorio del notebook: {notebook_dir}")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
from tqdm.auto import tqdm
import brian2

# Configurar Brian2 para usar numpy (evita errores de compilación en C++ y problemas con SymPy/Cython)
brian2.prefs.codegen.target = "numpy"

# Nuestros módulos
try:
    from main import FraudDetectionPipeline, generate_synthetic_transactions
    from encoders import RateEncoder, timeralEncoder, PopulationEncoder, TransactionEncoder
    from models_snn import FraudSNN, demonstrate_lif_neuron  # type: ignore[attr-defined]
    print("✅ Importación de módulos del proyecto completada!")
except ImportError as e:
    print(f"⚠️ Error al importar módulos: {e}")
    print(f"  sys.path: {sys.path[:3]}")

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 2. Generación de Datos Sintéticos

Vamos a crear un conjunto de datos sintético de transacciones bancarias con patrones realistas.

In [ ]:
# Generar 1000 transacciones (5% fraudes)
print("Generando transacciones sintéticas...")
df = generate_synthetic_transactions(n=1000, fraud_ratio=0.05)

print(f"\n📊 Conjunto de datos generado:")
print(f"  Total de transacciones: {len(df)}")
print(f"  Transacciones legítimas: {np.sum(df['is_fraud'] == 0)}")
print(f"  Transacciones fraudulentas: {np.sum(df['is_fraud'] == 1)}")
print(f"  Tasa de fraude: {df['is_fraud'].mean():.2%}")

# Mostrar primeras filas
df.head(10)

In [ ]:
# Visualizar distribución de valores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución de valores por clase
df[df['is_fraud'] == 0]['amount'].hist(bins=50, alpha=0.7, label='Legítima', ax=axes[0], color='green')
df[df['is_fraud'] == 1]['amount'].hist(bins=30, alpha=0.7, label='Fraude', ax=axes[0], color='red')
axes[0].set_xlabel('Valor de transacción ($)')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de valores por clase')
axes[0].legend()

# Frecuencia diaria por clase
df.boxplot(column='daily_frequency', by='is_fraud', ax=axes[1])
axes[1].set_xlabel('Clase (0=Legítima, 1=Fraude)')
axes[1].set_ylabel('Frecuencia diaria')
axes[1].set_title('Frecuencia de transacciones por clase')
plt.suptitle('')

plt.tight_layout()
plt.show()

print("\n📊 Patrones observados:")
print("  - Las fraudes tienden a tener valores más altos")
print("  - Las fraudes tienen mayor frecuencia de transacciones")

## 3. Codificación de Picos

Demostrar cómo las características de la transacción se convierten en picos temporales.

In [ ]:
# Ejemplo de codificación por tasa
print("=== CODIFICACIÓN POR TASA ===")
print("Codifica valores continuos como frecuencia de picos\n")

rate_encoder = RateEncoder(min_rate=1, max_rate=100, duration=0.1)

# Prueba con diferentes valores
test_amounts = [100, 500, 1000, 5000, 10000]

fig, axes = plt.subplots(len(test_amounts), 1, figsize=(12, 10))

for idx, amount in enumerate(test_amounts):
    spike_times = rate_encoder.encode(amount, min_val=0, max_val=10000)
    
    # Visualización
    if spike_times:
        axes[idx].eventplot([spike_times], linewidths=2, colors='blue')
        axes[idx].set_xlim(0, 0.1)
        axes[idx].set_ylim(0.5, 1.5)
        axes[idx].set_ylabel(f'$ {amount}')
        axes[idx].set_title(f'valor: $ {amount} → {len(spike_times)} picos')
        axes[idx].grid(True, alpha=0.3)
    
axes[-1].set_xlabel('Tiempo (segundos)')
plt.suptitle('Codificación por tasa: Valor → Frecuencia de picos', fontsize=14, y=1.0)
plt.tight_layout()
plt.show()

print("\n💡 Nota: Valores más altos generan más picos (mayor frecuencia)")

In [ ]:
# Ejemplo de codificación por población (Geolocalización)
print("=== CODIFICACIÓN POR POBLACIÓN ===")
print("Codifica valores usando múltiples neuronas con campos receptivos\n")

pop_encoder = PopulationEncoder(n_neurons=20, min_val=-1, max_val=1, sigma=0.15)

# Prueba con diferentes ubicaciones
test_locations = [-0.8, -0.3, 0.0, 0.4, 0.9]

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Gráfico 1: Activación de neuronas
for loc in test_locations:
    activations = np.exp(-((pop_encoder.centers - loc) ** 2) / (2 * pop_encoder.sigma ** 2))
    axes[0].plot(pop_encoder.centers, activations, marker='o', label=f'Ubicación = {loc:.1f}', alpha=0.7)

axes[0].set_xlabel('Centro de neurona')
axes[0].set_ylabel('Activación')
axes[0].set_title('Activación de la población neuronal por ubicación')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Raster plot de picos
spike_data = []
for idx, loc in enumerate(test_locations):
    encoding = pop_encoder.encode(loc, duration=0.1)
    if len(encoding.spike_times) > 0:
        for t, n in zip(encoding.spike_times, encoding.neuron_indices):
            spike_data.append([t, n + idx * 25]) # Desplazamiento para visualización

if spike_data:
    spike_array = np.array(spike_data)
    axes[1].scatter(spike_array[:, 0], spike_array[:, 1], marker='|', s=100, alpha=0.6)

axes[1].set_xlabel('Tiempo (segundos)')
axes[1].set_ylabel('Neurona + desplazamiento')
axes[1].set_title('Picos generados por la población neuronal')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Nota: Cada ubicación activa un grupo diferente de neuronas")

## 4. Arquitectura SNN

Visualizar y comprender la arquitectura de la Red Neuronal de Picos.

In [ ]:
# Demostración de neurona LIF individual
print("=== NEURONA LIF (FUGA INTEGRAR-DISPARAR) ===")
print("Demostración del comportamiento de la neurona LIF\n")

lif_data = demonstrate_lif_neuron()  # type: ignore[name-defined]

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Gráfico 1: Corriente de entrada
axes[0].plot(lif_data['time'], lif_data['input'], color='blue', linewidth=2)
axes[0].set_ylabel('Corriente de entrada (I)')
axes[0].set_title('Estímulo de entrada (corriente escalón)')
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Potencial de membrana y picos
axes[1].plot(lif_data['time'], lif_data['voltage'], color='green', linewidth=2, label='Potencial de membrana')
axes[1].axhline(-50, color='red', linestyle='--', label='Umbral (-50mV)', alpha=0.7)
axes[1].axhline(-70, color='gray', linestyle='--', label='Reposo (-70mV)', alpha=0.5)

# Marcar picos
for spike_time in lif_data['spikes']:
    axes[1].axvline(spike_time, color='red', alpha=0.3, linewidth=1)

axes[1].set_xlabel('Tiempo (ms)')
axes[1].set_ylabel('Voltaje (mV)')
axes[1].set_title(f'Potencial de membrana (total de {len(lif_data["spikes"])} picos)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Análisis:")
print(f"  - Picos detectados: {len(lif_data['spikes'])}")
print(f"  - Frecuencia media: {len(lif_data['spikes']) / (lif_data['time'][-1] / 1000):.1f} Hz")
print(f"  - ISI medio: {np.mean(np.diff(lif_data['spikes'])):.2f} ms" if len(lif_data['spikes']) > 1 else "")

In [ ]:
# Estadísticas de arquitectura SNN
print("=== ARQUITECTURA SNN ===")

snn = FraudSNN(input_size=256, hidden_sizes=[128, 64], output_size=2)
stats = snn.get_network_stats()  # type: ignore[attr-defined]

print(f"\n📊 Estructura de la red:")
print(f"  Capa de entrada: {stats['layers']['input']} neuronas")
print(f"  Capa oculta 1: {stats['layers']['hidden'][0]} neuronas (LIF)")
print(f"  Capa oculta 2: {stats['layers']['hidden'][1]} neuronas (LIF)")
print(f"  Capa de salida: {stats['layers']['output']} neuronas")
print(f"\n  Total de neuronas: {stats['total_neurons']}")
print(f"  Total de sinapsis: {stats['total_synapses']}")

print(f"\n⚖️ Pesos sinápticos:")
print(f"  Media: {stats['weights']['mean']:.4f}")
print(f"  Desviación estándar: {stats['weights']['std']:.4f}")
print(f"  Mín: {stats['weights']['min']:.4f}")
print(f"  Máx: {stats['weights']['max']:.4f}")

# Visualizar arquitectura
layer_sizes = [256, 128, 64, 2]
layer_names = ['Entrada\n(256)', 'Oculta 1\n(128)', 'Oculta 2\n(64)', 'Salida\n(2)']

fig, ax = plt.subplots(figsize=(14, 6))

# Dibujar capas
x_positions = np.linspace(0, 10, len(layer_sizes))
max_size = max(layer_sizes)

for i, (size, name, x) in enumerate(zip(layer_sizes, layer_names, x_positions)):
    y_positions = np.linspace(0, max_size, size)
    
    # Limitar visualización para capas grandes
    display_neurons = min(size, 20)
    y_display = np.linspace(0, max_size, display_neurons)
    
    ax.scatter([x] * display_neurons, y_display, s=100, alpha=0.7, 
               color=f'C{i}', label=name, zorder=3)
    
    # Conectar con la siguiente capa
    if i < len(layer_sizes) - 1:
        next_x = x_positions[i + 1]
        next_size = min(layer_sizes[i + 1], 20)
        next_y = np.linspace(0, max_size, next_size)
        
        # Dibujar algunas conexiones (muestra)
        for y1 in y_display[::3]:
            for y2 in next_y[::3]:
                ax.plot([x, next_x], [y1, y2], 'k-', alpha=0.05, linewidth=0.5, zorder=1)

ax.set_xlim(-1, 11)
ax.set_ylim(-20, max_size + 20)
ax.set_xticks(x_positions)
ax.set_xticklabels(layer_names)
ax.set_yticks([])
ax.set_title('Arquitectura de red neuronal de picos para detección de fraude', fontsize=14)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.tight_layout()
plt.show()

## 5. Pipeline Completo

Ejecutar el pipeline de extremo a extremo: entrenar y evaluar.

In [ ]:
# Inicializar pipeline
print("Inicializando pipeline de detección de fraude...")
pipeline = FraudDetectionPipeline()

# Dividir entrenamiento/prueba - usar datos más pequeños para una demostración rápida
train_size = int(0.8 * len(df))
train_data = df[:train_size].copy()
test_data = df[train_size:].copy()

# Para la demo, usar solo un subconjunto pequeño de entrenamiento
# Reducido a 20 muestras para ejecución rápida en modo numpy (sin compilación C++)
train_subset_size = min(20, len(train_data))
train_subset = train_data.sample(n=train_subset_size, random_state=42)

print(f"\n División de los datos:")
print(f" Entrenamiento (subconjunto para demo): {len(train_subset)} transacciones")
print(f" Prueba: {len(test_data)} transacciones")
print(f"\n Nota: Usando subconjunto reducido para demostración rápida")

print("\n⏳ Iniciando entrenamiento con STDP...")
print("(Usando pocas épocas y datos reducidos para una demostración rápida)\n")

# Reducir drásticamente para la demo
epochs = 2
print(f" Entrenamiento: {epochs} épocas con {len(train_subset)} transacciones")

# Entrenamiento rápido
start_time = time.time()
print("Entrenando...")
pipeline.train(train_subset, epochs=epochs)
training_time = time.time() - start_time

print(f"\n Entrenamiento finalizado en {training_time:.1f}s")
print(f" tiempo medio por época: {training_time/epochs:.2f}s")
print(f" tasa: {len(train_subset) * epochs / training_time:.1f} transacciones/s")

In [ ]:
# Evaluar en el conjunto de prueba
print(" Evaluando el modelo en el conjunto de prueba...")
print(f"total de {len(test_data)} transacciones\n")

# Evaluación con tiempo estimado
start_eval = time.time()
metrics = pipeline.evaluate(test_data)
eval_time = time.time() - start_eval

print(f"\n Evaluación finalizada en {eval_time:.2f}s")
print(f" velocidad: {len(test_data)/eval_time:.1f} transacciones/s")


In [ ]:
# Visualizar matriz de confusión
from sklearn.metrics import confusion_matrix
import numpy as np

# Convertir explícitamente a arrays de numpy
y_true = test_data['is_fraud'].to_numpy()
y_pred = []

print(" Generando predicciones para la matriz de confusión...")
for _, row in tqdm(test_data.iterrows(), total=len(test_data), desc="Predicciones", unit="txn"):
    result = pipeline.predict(row.to_dict())
    y_pred.append(int(result['is_fraud']))

# Convertir y_pred a array de numpy
y_pred = np.array(y_pred, dtype=int)

cm = confusion_matrix(y_true, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Legítima', 'Fraude'],
            yticklabels=['Legítima', 'Fraude'],
            cbar_kws={'label': 'Conteo'})
ax.set_xlabel('Predicción')
ax.set_ylabel('Real')
ax.set_title('Matriz de confusión - Detección de fraude neuromórfica')
plt.tight_layout()
plt.show()

print(f"\n Exactitud: {metrics['accuracy']:.2%}")
print(f"  Precisión: {metrics['precision']:.2%}")
print(f"  Sensibilidad (recall): {metrics['recall']:.2%}")
print(f"  F1-Score: {metrics['f1_score']:.2%}")

## 6. Ejemplos de Predicción Individual

Probar con transacciones específicas.

In [ ]:
# Ejemplo 1: Transacción legítima típica
print("=" * 60)
print("Ejemplo 1: Transacción Legítima")
print("=" * 60)

legitimate_txn = {
 'id': 'demo_001',
 'amount': 85.50,
 'timestamp': time.time(),
 'merchant_category': 'groceries',
 'location': (-23.5505, -46.6333), # São Paulo
 'device_id': 'device_regular_001',
 'daily_frequency': 3
}

result = pipeline.predict(legitimate_txn)

print(f"\nTransacción:")
print(f" valor: $ {legitimate_txn['amount']:.2f}")
print(f" Categoría: {legitimate_txn['merchant_category']}")
print(f" Ubicación: São Paulo")

print(f"\n Resultado del análisis:")
print(f" Fraude detectada: {' SÍ' if result['is_fraud'] else ' NO'}")
print(f" Confianza: {result['confidence']:.2%}")
print(f" Puntuación Legítima: {result['legitimate_score']:.2f} Hz")
print(f" Puntuación Fraude: {result['fraud_score']:.2f} Hz")
print(f" Latencia: {result['latency_ms']:.2f}ms")
print(f" Picos generados: {result['n_spikes_generated']}")

In [ ]:
# Ejemplo 2: Transacción sospechosa (alta probabilidad de fraude)
print("=" * 60)
print("Ejemplo 2: Transacción Sospechosa")
print("=" * 60)

suspicious_txn = {
 'id': 'demo_002',
 'amount': 8500.00, # valor alto
 'timestamp': time.time(),
 'merchant_category': 'electronics',
 'location': (51.5074, -0.1278), # Londres (ubicación poco común)
 'device_id': 'device_new_unknown', # Dispositivo nuevo
 'daily_frequency': 25 # frecuencia anormal
}

result = pipeline.predict(suspicious_txn)

print(f"\nTransacción:")
print(f" valor: $ {suspicious_txn['amount']:.2f}")
print(f" Categoría: {suspicious_txn['merchant_category']}")
print(f" Ubicación: Londres (poco común)")
print(f" Dispositivo: nuevo/desconocido")

print(f"\n Resultado del análisis:")
print(f" Fraude detectada: {' SÍ' if result['is_fraud'] else ' NO'}")
print(f" Confianza: {result['confidence']:.2%}")
print(f" Puntuación Legítima: {result['legitimate_score']:.2f} Hz")
print(f" Puntuación Fraude: {result['fraud_score']:.2f} Hz")
print(f" Latencia: {result['latency_ms']:.2f}ms")
print(f" Picos generados: {result['n_spikes_generated']}")

if result['is_fraud']:
 print(f"\n ALERTA: ¡Transacción bloqueada para análisis manual!")

## 7. Análisis de Rendimiento

Evaluar latencia y rendimiento del sistema.

In [ ]:
# Benchmark de latencia
print("=== Benchmark de latencia ===")
n_samples = min(100, len(test_data))
print(f"Probando {n_samples} transacciones...\n")

latencies = []
sample_txns = test_data.sample(n=n_samples)

for _, row in tqdm(sample_txns.iterrows(), total=n_samples, desc="Benchmark", unit="txn"):
 start = time.time()
 result = pipeline.predict(row.to_dict())
 latency = (time.time() - start) * 1000 # ms
 latencies.append(latency)

latencies = np.array(latencies)

print(f"\n Estadísticas de latencia:")
print(f" media: {latencies.mean():.2f}ms")
print(f" mediana: {np.median(latencies):.2f}ms")
print(f" Mín: {latencies.min():.2f}ms")
print(f" Máx: {latencies.max():.2f}ms")
print(f" P95: {np.percentile(latencies, 95):.2f}ms")
print(f" P99: {np.percentile(latencies, 99):.2f}ms")

throughput = 1000 / latencies.mean() # transacciones por segundo
print(f"\n Rendimiento estimado: {throughput:.0f} transacciones/s")

# Visualizar distribución
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(latencies, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].axvline(latencies.mean(), color='red', linestyle='--', label=f'media: {latencies.mean():.2f}ms')
axes[0].axvline(np.percentile(latencies, 95), color='orange', linestyle='--', label=f'P95: {np.percentile(latencies, 95):.2f}ms')
axes[0].set_xlabel('latencia (ms)')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de latencia')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(latencies, vert=True)
axes[1].set_ylabel('latencia (ms)')
axes[1].set_title('Diagrama de caja de la latencia')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 8. Conclusiones

### Ventajas de la aproximación neuromórfica

1. **Ultrabaja latencia**: Detección en ~10ms
2. **Procesamiento temporal nativo**: Captura patrones de secuencia de forma natural
3. **Eficiencia energética**: Ideal para despliegue en dispositivos de borde
4. **Aprendizaje biológico**: STDP permite adaptación continua

### Aplicaciones en bancos y fintechs

- Detección de fraude en tiempo real en POS
- Protección de transacciones Pix/TED/DOC
- Monitoreo de carteras digitales
- Análisis comportamental en banca móvil

### Próximos pasos

- Despliegue en hardware neuromórfico (Intel Loihi, IBM TrueNorth)
- Integración con sistemas heredados vía API
- Explicabilidad (SHAP para SNNs)
- Aprendizaje federado entre instituciones

---

**Autor:** Mauro Risonho de Paula Assumpção 
**Proyecto:** Computación neuromórfica para ciberseguridad bancaria